# Data validation

In [9]:
import boto3
import awswrangler as wr
import pandas as pd
import pandera as pa

We load the dataset as this makes it easier to set our expectations

In [12]:
boto3.setup_default_session()
ssm = boto3.client('ssm')
parameter = ssm.get_parameter(Name='/datafy-samples/bucket/name')
bucket = parameter['Parameter']['Value']

df = wr.s3.read_csv(path=f"s3://{bucket}/titanic/raw/train.csv")
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


We describe the expected input and add some checks

In [22]:
schema = pa.DataFrameSchema({
    "PassengerId": pa.Column(int),
    "Survived": pa.Column(int, checks=pa.Check.isin([0,1])),
    "Pclass": pa.Column(int, checks=pa.Check.isin([0,1,2,3])),
    "Name": pa.Column(str),
    "Sex": pa.Column(str, checks=pa.Check(lambda s: s.isin(["male", "female"]))),
    "Age": pa.Column(float, checks=pa.Check.less_than(100, ignore_na=True), nullable=True),
    "SibSp": pa.Column(int),
    "Parch": pa.Column(int),
    "Ticket": pa.Column(str),
    "Fare": pa.Column(float),
    "Cabin": pa.Column(str, nullable=True),
    "Embarked": pa.Column(str, nullable=True),
}, strict=False)

Next, we can validate our dataframe against this schema

In [23]:
schema(df)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C
